<a href="https://colab.research.google.com/github/Junaidkhan1485786/Pinecone-Introduction/blob/main/pinconeVectordatabase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU \
    openai==0.27.7 \
    pinecone-client==3.1.0 \
    pinecone-datasets==0.7.0 \
    tqdm \
    pinecone-notebooks==0.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 65.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidi

In [2]:
from pinecone_datasets import load_dataset

dataset = load_dataset('youtube-transcripts-text-embedding-ada-002')
# we drop sparse_values as they are not needed for this example
dataset.documents.drop(['metadata'], axis=1, inplace=True)
dataset.documents.rename(columns={'blob': 'metadata'}, inplace=True)
dataset.head()

,id,values,sparse_values,metadata
0,35Pdoyi6ZoQ-t0.0,"[-0.010402066633105278, -0.018359748646616936,...",None,"{'channel_id': 'UCv83tO5cePwHMt1952IVVHw', 'en..."
1,35Pdoyi6ZoQ-t18.48,"[-0.011849376372992992, 0.0007984379190020263,...",None,"{'channel_id': 'UCv83tO5cePwHMt1952IVVHw', 'en..."
2,35Pdoyi6ZoQ-t32.36,"[-0.014534404501318932, -0.0003158661129418760...",None,"{'channel_id': 'UCv83tO5cePwHMt1952IVVHw', 'en..."
3,35Pdoyi6ZoQ-t51.519999999999996,"[-0.011597747914493084, -0.007550035137683153,...",None,"{'channel_id': 'UCv83tO5cePwHMt1952IVVHw', 'en..."
4,35Pdoyi6ZoQ-t67.28,"[-0.015879768878221512, 0.0030445053707808256,...",None,"{'channel_id': 'UCv83tO5cePwHMt1952IVVHw', 'en..."


In [3]:
# create index for applying pinecone

In [5]:
import os

if not os.environ.get("pine cone api key entyer here"):
    from pinecone_notebooks.colab import Authenticate
    Authenticate()

In [6]:
from pinecone import Pinecone

api_key = os.environ.get("pine cone api key entyer here")

# configure client
pc = Pinecone(api_key=api_key)

In [7]:
# Now we setup our index specification, this allows us to define the cloud provider and region where we want to deploy our index

from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

In [8]:
index_name = 'gen-qa-openai-fast'

In [9]:
# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=1536,  # dimensionality of text-embedding-ada-002
        metric='cosine',
        spec=spec
    )
# connect to index
index = pc.Index(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 39250}},
 'total_vector_count': 39250}

In [10]:
# We can see the index is currently empty with a total_vector_count of 0. We can begin populating it with OpenAI text-embedding-ada-002 built embeddings like so:

for batch in dataset.iter_documents(batch_size=100):
    index.upsert(batch)

In [21]:
import openai

# get api key from platform.openai.com
openai.api_key = os.getenv('OPENAI_API_KEY') or 'open_api key enter here' # above use picone api, here use openapi(chatgpt) api

embed_model = "text-embedding-ada-002"

In [22]:
query = (
    "Which training method should I use for sentence transformers when " +
    "I only have pairs of related sentences?"
)

res = openai.Embedding.create(
    input=[query],
    engine=embed_model
)

# retrieve from Pinecone
xq = res['data'][0]['embedding']

# get relevant contexts (including the questions)
res = index.query(vector=xq, top_k=2, include_metadata=True)

RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

In [24]:
res

# {'matches': [{'id': 'pNvujJ1XyeQ-t418.88',
#               'metadata': {'channel_id': 'UCv83tO5cePwHMt1952IVVHw',
#                            'end': 568.0,
#                            'published': '2021-11-24 16:24:24 UTC',
#                            'start': 418.0,
#                            'text': 'pairs of related sentences you can go '
#                                    'ahead and actually try training or '
#                                    'fine-tuning using NLI with multiple '
#                                    "negative ranking loss. If you don't have "
#                                    'that fine. Another option is that you have '
#                                    'a semantic textual similarity data set or '
#                                    'STS and what this is is you have so you '
#                                    'have sentence A here, sentence B here and '
#                                    'then you have a score from from 0 to 1 '
#                                    'that tells you the similarity between '
#                                    'those two scores and you would train this '
#                                    'using something like cosine similarity '
#                                    "loss. Now if that's not an option and your "
#                                    'focus or use case is on building a '
#                                    'sentence transformer for another language '
#                                    'where there is no current sentence '
#                                    'transformer you can use multilingual '
#                                    'parallel data. So what I mean by that is '
#                                    'so parallel data just means translation '
#                                    'pairs so if you have for example a English '
#                                    'sentence and then you have another '
#                                    'language here so it can it can be anything '
#                                    "I'm just going to put XX and that XX is "
#                                    'your target language you can fine-tune a '
#                                    'model using something called multilingual '
#                                    'knowledge distillation and what that does '
#                                    'is takes a monolingual model for example '
#                                    'in English and using those translation '
#                                    'pairs it distills the knowledge the '
#                                    'semantic similarity knowledge from that '
#                                    'monolingual English model into a '
#                                    'multilingual model which can handle both '
#                                    'English and your target language. So '
#                                    "they're three options quite popular very "
#                                    'common that you can go for and as a '
#                                    'supervised methods the chances are that '
#                                    'probably going to outperform anything you '
#                                    'do with unsupervised training at least for '
#                                    'now. So if none of those sound like '
#                                    'something',
#                            'title': 'Today Unsupervised Sentence Transformers, '
#                                     'Tomorrow Skynet (how TSDAE works)',
#                            'url': 'https://youtu.be/pNvujJ1XyeQ'},
#               'score': 0.865291,
#               'values': []},
#              {'id': 'WS1uVMGhlWQ-t747.92',
#               'metadata': {'channel_id': 'UCv83tO5cePwHMt1952IVVHw',
#                            'end': 906.0,
#                            'published': '2021-10-20 17:06:20 UTC',
#                            'start': 747.0,
#                            'text': "pooling approach. Or we can't use it in "
#                                    'its current form. Now the solution to this '
#                                    'problem was introduced by two people in '
#                                    '2019 Nils Reimers and Irenia Gurevich. '
#                                    'They introduced what is the first sentence '
#                                    'transformer or sentence BERT. And it was '
#                                    'found that sentence BERT or S BERT '
#                                    'outformed all of the previous Save the Art '
#                                    'models on pretty much all benchmarks. Not '
#                                    'all of them but most of them. And it did '
#                                    'it in a very quick time. So if we compare '
#                                    'it to BERT, if we wanted to find the most '
#                                    'similar sentence pair from 10,000 '
#                                    'sentences in that 2019 paper they found '
#                                    'that with BERT that took 65 hours. With S '
#                                    'BERT embeddings they could create all the '
#                                    'embeddings in just around five seconds. '
#                                    'And then they could compare all those with '
#                                    "cosine similarity in 0.01 seconds. So it's "
#                                    'a lot faster. We go from 65 hours to just '
#                                    'over five seconds which is I think pretty '
#                                    "incredible. Now I think that's pretty much "
#                                    'all the context we need behind sentence '
#                                    'transformers. And what we do now is dive '
#                                    'into a little bit of how they actually '
#                                    'work. Now we said before we have the core '
#                                    'transform models and what S BERT does is '
#                                    'fine tunes on sentence pairs using what is '
#                                    'called a Siamese architecture or Siamese '
#                                    'network. What we mean by a Siamese network '
#                                    'is that we have what we can see, what can '
#                                    'view as two BERT models that are identical '
#                                    'and the weights between those two models '
#                                    'are tied. Now in reality when implementing '
#                                    'this we just use a single BERT model. And '
#                                    'what we do is we process one sentence, a '
#                                    'sentence A through the model and then we '
#                                    'process another sentence, sentence B '
#                                    "through the model. And that's the sentence "
#                                    'pair. So with our cross-linked we were '
#                                    'processing the sentence pair together. We '
#                                    'were putting them both together, '
#                                    'processing them all at once. This time we '
#                                    'process them separately. And during '
#                                    'training what happens is the weights '
#                                    'within BERT are optimized to reduce the '
#                                    'difference between two vector embeddings '
#                                    'or two sentence',
#                            'title': 'Intro to Sentence Embeddings with '
#                                     'Transformers',
#                            'url': 'https://youtu.be/WS1uVMGhlWQ'},
#               'score': 0.86448735,
#               'values': []}],
#  'namespace': '',
#  'usage': {'read_units': 6}}

NameError: name 'res' is not defined

In [ ]:
limit = 3750

import time

def retrieve(query):
    res = openai.Embedding.create(
        input=[query],
        engine=embed_model
    )

    # retrieve from Pinecone
    xq = res['data'][0]['embedding']

    # get relevant contexts
    contexts = []
    time_waited = 0
    while (len(contexts) < 3 and time_waited < 60 * 12):
        res = index.query(vector=xq, top_k=3, include_metadata=True)
        contexts = contexts + [
            x['metadata']['text'] for x in res['matches']
        ]
        print(f"Retrieved {len(contexts)} contexts, sleeping for 15 seconds...")
        time.sleep(15)
        time_waited += 15

    if time_waited >= 60 * 12:
        print("Timed out waiting for contexts to be retrieved.")
        contexts = ["No contexts retrieved. Try to answer the question yourself!"]


    # build our prompt with the retrieved contexts included
    prompt_start = (
        "Answer the question based on the context below.\n\n"+
        "Context:\n"
    )
    prompt_end = (
        f"\n\nQuestion: {query}\nAnswer:"
    )
    # append contexts until hitting limit
    for i in range(1, len(contexts)):
        if len("\n\n---\n\n".join(contexts[:i])) >= limit:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts[:i-1]) +
                prompt_end
            )
            break
        elif i == len(contexts)-1:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts) +
                prompt_end
            )
    return prompt


def complete(prompt):
    # instructions
    sys_prompt = "You are a helpful assistant that always answers questions."
    # query text-davinci-003
    res = openai.ChatCompletion.create(
        model='gpt-3.5-turbo-0613',
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )
    return res['choices'][0]['message']['content'].strip()

In [26]:
# # first we retrieve relevant items from Pinecone
query_with_contexts = retrieve(query)
query_with_contexts


# # Answer the question based on the context below.

# # Context:
# # pooling approach. Or we can't use it in its current form. Now the solution to this problem was introduced by two people in 2019 Nils Reimers and Irenia Gurevich. They introduced what is the first sentence transformer or sentence BERT. And it was found that sentence BERT or S BERT outformed all of the previous Save the Art models on pretty much all benchmarks. Not all of them but most of them. And it did it in a very quick time. So if we compare it to BERT, if we wanted to find the most similar sentence pair from 10,000 sentences in that 2019 paper they found that with BERT that took 65 hours. With S BERT embeddings they could create all the embeddings in just around five seconds. And then they could compare all those with cosine similarity in 0.01 seconds. So it's a lot faster. We go from 65 hours to just over five seconds which is I think pretty incredible. Now I think that's pretty much all the context we need behind sentence transformers. And what we do now is dive into a little bit of how they actually work. Now we said before we have the core transform models and what S BERT does is fine tunes on sentence pairs using what is called a Siamese architecture or Siamese network. What we mean by a Siamese network is that we have what we can see, what can view as two BERT models that are identical and the weights between those two models are tied. Now in reality when implementing this we just use a single BERT model. And what we do is we process one sentence, a sentence A through the model and then we process another sentence, sentence B through the model. And that's the sentence pair. So with our cross-linked we were processing the sentence pair together. We were putting them both together, processing them all at once. This time we process them separately. And during training what happens is the weights within BERT are optimized to reduce the difference between two vector embeddings or two sentence

# # ---

# # pairs of related sentences you can go ahead and actually try training or fine-tuning using NLI with multiple negative ranking loss. If you don't have that fine. Another option is that you have a semantic textual similarity data set or STS and what this is is you have so you have sentence A here, sentence B here and then you have a score from from 0 to 1 that tells you the similarity between those two scores and you would train this using something like cosine similarity loss. Now if that's not an option and your focus or use case is on building a sentence transformer for another language where there is no current sentence transformer you can use multilingual parallel data. So what I mean by that is so parallel data just means translation pairs so if you have for example a English sentence and then you have another language here so it can it can be anything I'm just going to put XX and that XX is your target language you can fine-tune a model using something called multilingual knowledge distillation and what that does is takes a monolingual model for example in English and using those translation pairs it distills the knowledge the semantic similarity knowledge from that monolingual English model into a multilingual model which can handle both English and your target language. So they're three options quite popular very common that you can go for and as a supervised methods the chances are that probably going to outperform anything you do with unsupervised training at least for now. So if none of those sound like something

# # ---

# # We're going to be focusing on one specific training method which I think is quite useful because all it really needs is a reasonably small data set of parallel data which is simply translation pairs from a source language like English to whichever other language you're using. So obviously if you are wanting to train a sentence transformer in a language that doesn't really have that much data, it's particularly sentence similarity data, this can be really useful for actually taking a high performing, for example, English sentence transformer and transferring that knowledge or distilling that knowledge into a sentence transformer for your own language. So I think this will be pretty useful for a lot of you. And let's jump straight into it. Before we really get into the whole multilingual sentence transformer part of the video, I just want to sort of give an impression of what these multilingual sentence transformers are actually doing. So on here we can see a single English sentence or brief phrase down at the bottom, I love plants, and the rest of these are all in Italian. So what we have here are a vector representations of dense vector representations of these phrases. And a monolingual sentence transformer, which is most of the sentence transformers, will only cope with one language. So we would hope that phrases that have a similar meaning end up within the same sort of vector space. So like we have for amo lippiante here, and I love plants, these are kind of in the same space. A monolingual sentence transformer would do that for similar sentences. So in English, we might have I love plants and I like plants, which is actually what we have up here. So this here is Italian for I like plants. And we would hope that they're in a similar area, whereas irrelevant or

# # Question: Which training method should I use for sentence transformers when I only have pairs of related sentences?
# # Answer:

# evious Save the Art models on pretty much all benchmarks. Not all of them but most of them. And
# it did it in a very quick time. So if we compare it to BERT, if we wanted to find the most similar sentence pair fro
# m 10,000 sentences in that 2019 paper they found that with BERT that took 65 hours. With S BERT embeddings they could cre
# ate all the embeddings in just around five seconds. And then they could compare all those with cosine similarity in 0.01 se
# conds. So it's a lot faster. We go from 65 hours to just over five seconds which is I think pretty incredible. Now I think tha
# t's pretty much all the context we need behind sentence transformers. And what we do now is dive into a little bit of how they a
# ctually work. Now we said before we have the core transform models and what S BERT does is fine tunes on sentence pairs using wha
# t is called a Siamese architecture or Siamese network. What we mean by a Siamese network is that we have what we can see, what can
# view as two BERT models that are identical and the weights between those two models are tied. Now in reality when implementing this we just use a single BERT model. And what we do is we process one sentence, a sentence A through the model and then we process another sentence, sentence B through the model. And that's the sentence pair. So with our cross-linked we were processing the sentence pair together. We were putting them both together, processing them all at once. This time we process them separately. And during training what happens is the weights within BERT are optimized to reduce the difference between two vector embeddings or two sentence\n\n---\n\npairs of related sentences you can go ahead and actually try training or fine-tuning using NLI with multiple negative ranking loss. If you don't have that fine. Another option is that you have a semantic textual similarity data set or STS and what this is is you have so you have sentence A here, sentence B here and then you have a score from from 0 to 1 that tells you the similarity between those two scores and you would train this using something like cosine similarity loss. Now if that's not an option and your focus or use case is on building a sentence transformer for another language where there is no current sentence transformer you can use multilingual parallel data. So what I mean by that is so parallel data just means translation pairs so if you have for example a English sentence and then you have another language here so it can it can be anything I'm just going to put XX and that XX is your target language you can fine-tune a model using something called multilingual knowledge distillation and what that does is takes a monolingual model for example in English and using those translation pairs it distills the knowledge the semantic similarity knowledge from that monolingual English model into a multilingual model which can handle both English and your target language. So they're three options quite popular very common that you can go for and as a supervised methods the chances are that probably going to outperform anything you do with unsupervised training at least for now. So if none of those sound like something\n\n---\n\nWe're going to be focusing on one specific training method which I think is quite useful because all it really needs is a reasonably small data set of parallel data which is simply translation pairs from a source language
# like English to whichever other language you're using. So obviously if you are wanting to train a sentence transformer in a language that doesn't really have that much data, it's particularly sentence similarity data, this can be really useful for actually taking a high performing, for example, English sentence transformer and transferring that knowledge or distilling that knowledge into a sentence transformer for your own language. So I think this will be pretty useful for a lot of you. And let's jump straight into it. Before we really get into the whole multilingual sentence transformer part of the video, I just want to sort of give an impression of what these multilingual sentence transformers are actually doing. So on here we can see a single English sentence or brief phrase down at the bottom, I love plants, and the rest of these are all in Italian. So what we have here are a vector representations of dense vector representations of these phrases. And a monolingual sentence transformer, which is most of the sentence transformers, will only cope with one language. So we would hope that phrases that have a similar meaning end up within the same sort of vector space. So like we have for amo lippiante here, and I love plants, these are kind of in the same space. A monolingual sentence transformer would do that for similar sentences. So in English, we might have I love plants and I like plants, which is actually what we have up here. So this here is Italian for I like plants. And we would hope that they're in a similar area, whereas irrelevant or\n\nQuestion: Which training method should I use for sentence transformers when I only have pairs of related sentences?

NameError: name 'retrieve' is not defined

In [25]:
# # then we complete the context-infused query
complete(query_with_contexts)



# And we get a pretty great answer straight away, specifying to use multiple-rankings loss (also called multiple negatives ranking loss).

In [ ]:
pc.delete_index(index_name)